In [ ]:
class PruneTreeByMergingCentroids:
    __init__:
        Parameters:
            prefixString: It is a predefined string which will be used when generating cluster ids.
            cltree: The base cltree on which the pruning mechanism will be applied.
                    This has to be an instance of CLTree.
            balancedPrune: It is a boolean flag, value of which indicates if we want to create a "balanced" clusters
                           in terms of number of data points each cluster get.
                
        Operational Details:
                Suppose we have divided the whole customer space into 100 small groups by a CLTree and 
                each group at most has 5 data-points (This means the underlying CLTree has 100 leaves and
                each leaf at most has 5 data points). If we look at the customer data space graphically after
                CLTree has performed, we should realize that the whole customer space is divided by hyper rectangles
                and each rectangle is bounded by the cut which has been made to create that rectangle 
                (Classic decision tree and CLTree use the similar way to divide the whole data space gradually,
                 so if you search internet how decision tree divides the whole data space into numerous hyper 
                 rectangles, you should be able to find a fair amount of quality material and same logic applies 
                 here in the case of CLTree). Let’s say, we have a requirement that each cluster has to encompass
                at least 50 data points, otherwise it is too small to maintain. We have 100 small hyper-rectangles
                and each has at-most 5 data points each, and now we have a job at hand that we need to combine these
                100 hyper-rectangles gradually in a scientific way so that we can form groups of hyper-rectangles 
                where each group will be big enough to encompass at least 50 data points. 

                One of the better ways to represent a group of hyper-rectangles, when the goal is to measure how close
                they are from others in the space respectively, is to represent a rectangle by it’s centroid. 
                
                What is “centroid”?
                Refer to here: https://en.wikipedia.org/wiki/Centroid

                How to calculate centroid of a rectangle?
                Refer to here: https://www.engineeringintro.com/mechanics-of-structures/centre-of-gravity/centroid-of-rectangle/

                We have all the information needed to calculate centroids of all the hyper-rectangles. 
                How?
                Well, all the leaves are hyper-rectangle. 
                The boundary line of each hyper-rectangle at each dimension is represented by all the min and max
                of all the attributes of respectively of the underlying leaf. 
                Now, all there is left to do is calculation the centroid for each hyper rectangle represented by the
                leaves of the CLTree. 
                Once we get all the representation vectors (centroids of all the hyper-rectangle) we will compare
                the distance between one to other representation vectors and merge the pair together which are the 
                closest by the distance metric. After the merge of a pair, we recalculate the representation vector
                of the pair by calculating weighted average of the two representation vector  and then replace the
                individuals by the pair in the active participation group and continue the same process until our
                condition is stratifying metric and quality metric is met.  


    prune:
        Parameters:
            min_y: The "min_y" dictates the minimum number of data points required for a group to be 
                   considered as a cluster. This is an instance level variable. 
            searchMode: Boolean Flag, to indicate if the current pruning process, which is happending as part of some
                        grid search mechanism to calculate the best possible value for an hyper-parameter or not. 
                        If the value of this flag is False that means, we have already decided what would the values
                        of all the relevant hyper parameters and therefore have freedom to modify the structure 
                        of the underlying CLTree to optimize the search space.
                        If the value of the flag is True, we do not change the structure of the underlying CLTree, so
                        that we can reset all the modfications after calculating the quality metric at the end of the
                        process and revert back all changes made to the underlying CLTree and get back the original one,
                        which would be used for other values of the hyper parameter we are trying to optimize for.

        Operational Details: This is the driver, only external facing and wrapper function of the class. The pruning
                             process, whatever it may be, is controlled from here. 
                Algorithm:
                    1. Validate and set the value of instance level variable "min_y".
                    2. Initialize instance level variable "maxSoFar"(will be used to create unique cluster id)
                       and "version".
                    3. Initialize the some active instance level variables, totalDataInstancesbyEachGroup, 
                        finalRepresentationVectors, finalRepresentationVectorsToCluster, originalRepresentationVectors
                    4. Invoke, an internal function "intializeTouchedNodes" to set the "touching nodes" at each leaf
                       node level for the underlying CLTree. 
                    5. Invoke, an internal function "initializeCentroids" to get the original set of hyper-rectangle
                        centroids by each leaf node. [If there are some "touching nodes", the combination 
                        a leaf nodes and with all it's "touching nodes" appear once.].
                    6. Get the final "totalDataInstancesbyEachGroup", "finalRepresentationVectors",
                       "finalRepresentationVectorsToCluster" by invoking "pruningTreeByMergingRepresentationVectors" with
                       instance variable "originalRepresentationVectors" and "min_y". 
                    7. Measure the final cluster statistics by invoking "clusterStatistics" function with 
                       instance level "originalRepresentationVectors", "finalRepresentationVectors",
                       "finalRepresentationVectorsToCluster", "totalDataInstancesbyEachGroup" and store it in "result". 
                    8. If we are not currently searching of an optimized value for an hyper parameter and had decided
                       on the values of all the parameters, then we are free to "prune" the underlying CLTree to 
                       optimize the search space. The optimization of the search space for the cost of little
                       modificantion of the underlying CLTree doesn't change the result or any details other than 
                       may be depth/hight of some nodes where applicable. The process is performed by invoking 
                       "pruningRedundantNodes" with the "root" of the uinderlying CLTree. 
                    9. Return the final result to the calling function.
        
    identifyingTouchedRegions:
        Parameters:
            root: An instance of CLNode which is root of a CLTree instance.
                
        Operational Details:
            Defination:
                **This operation is only valid for a CLTree.**
                In a complex situation, it is possible that a broadly similar group of points got splited
                into several regions either because the group is bounded by an irregular shape or because
                in order to isolate one or more sparse region(s) from the original the underlying region,
                it is cut into more than one piece.
                
                A region, Y1, is said to touch another region, Y2, on the i-th dimension on the lower bounding
                surface (or the upper bounding surface), if they meet on the i-th dimension and intersect on all
                other dimensions. That is,
                1. max(Y1, i) = min(Y2, i) (or max(Y2, i) = min(Y1, i)) [for all i in cols./attrs. of data points]
                2. And min(Y1, j) < max(Y2, j) and max(Y1, j) > min(Y2, j) [for all j ≠ i]
                
                **Algorithm**:
                    1. Get the "preordered" list of leaves. [We are only concern about the leaves because at the end
                                                          these are the only set of nodes which contains all the
                                                          data points]
                    2. Initialize "mergeList", a blank list which will hold the pair of leaves which are touching
                                                            each-other by the above mentioned criterion.
                    3. for node_preOrderedPos in range(len(PreOrderedList_of_leaves)):
                        i. node = PreOrderedList_of_leaves[node_preOrderedPos]
                        ii. get the min and max of all the attribute values of the node.
                        iii. for all nxt_node in PreOrderedList_of_leaves[node_preOrderedPos:]:
                            a. for each attr_i in attributes:
                                **check for criteria 1. of the defination, if satisfies proceed to the step "a)"**
                                    a) for each attr_j in attributes except attr_i:
                                        **check for criteria 2. of the defination**
                            b. if both criteria 1 and 2 are satisfied a pair of leaf node:
                                a) Add the pair (node, nxt_node) to "mergeList".
                            
                            end for
                        
                        end for
                        
                    4. Declare an empty dictionary named "touchingNodes", which will contain standarized list of
                            toching nodes for each node if a node is touching atleast one other node. 
                    5. if "mergeList" is not empty:
                        a. invoke "normalizingMergeList()" with "mergeList",
                                            it will return the content of "touchingNodes"
                    6. Return "touchingNodes" to the calling function.
                                
                                    
    
    normalizingMergeList:
        Parameters: 
            mergeList: list of pair of nodes which are touching each-other by defination of "touching nodes".
                
        Operational Details:
            Create and return standarize list of touched nodes by each nodes respectively. This
            function takes care of "Commutative" and "Transitive" property to create standarize list of
            "touching nodes" for each node. 
            Ex.: mergeList: [(A,B), (C,D)] ==> touchingNodes: {A: [B], B: [A], C: [D], D:[C]}
                 mergeList: [(A,B), (B,C)] ==> touchingNodes: {A: [B, C], B: [A, C], C: [A, B]}
                        
            returns touchingNodes to the calling function.
            
    
    
    calcModifiedDataLengthForEachnode:
        Parameters: 
            node: An instance of CLNode (which is part of a CLTree instance).
                
        Operational Details:
            This function initializes the "modifiedLength" property/attribute of CLNode.
            After initializing the "touching nodes" for each node, the idea is the node itself and all the
            touching node will **virtually** be considered a sigle node even though they are physically apart.
            So if we want to know how many data points are there in any node at any given point of time, we
            should include the length of data of "touching nodes" as well as virtually they are part of the
            same node. 
            
            This is a recursive function and operation on the CLTree is "post-order"[processing the children
            first, then the node iself at any node] in nature.
            
            Algorithm:
                1. if the node is Leaf:
                    a. modfiedDataLength = own data length + ∑(data length of all "touching nodes")
                    b. node.modifiedDataLength = modifiedDataLength
                    c. return to the calling function
                2. else:
                    a. calculate the modifided data length of the left child.
                    b. calculate the modifided data length of the right child.
                    c. modfiedDataLength = left child modified data length + right child modified data length
                    d. node.modifiedDataLength = modifiedDataLength
                    e. return to the calling function
                
            This function set the values at the node level and doesn't return any value to the calling function.
            
    intializeTouchedNodes:
        Parameters:
            root: An instance of CLNode which is root of a CLTree instance.
                
        Operational Details:
            This function is driver for getting a standrized list of "touching nodes" by each leaf nodes and set 
            the values for touchedNodes, modifiedLength at the CLNode instance level. 
            [Refer to "identifyingTouchedRegions" to get a clear idea about "touching nodes"]
            Algorithm:
                1. get the standrized list of "touching nodes" by each leaf nodes by invoking
                                                                            "identifyingTouchedRegions".
                2. Assign the list of "touching nodes" by each leaf nodes at CLNode instance level one by one. 
                3. calculate and initialize "modifiedLength" property/attribute of each CLNode by
                                                             invoking "calcModifiedDataLengthForEachnode".
            
            This function set the values at the CLNode instance level and
                                                doesn't return any value to the calling function.    
        

    _accountForTouchedNodesForCentroids:
        Parameters:
            preOrderedListofLeaves: A list of leaves which was generated by traversing the initial CLTree in preorder.
            hyperRectangleCentroids: A python dictionary containing the initial centroid vector for each hyperrectangle
                                     created by each leaf.
            attributes: List of all the attributes in a constant order.
                
        Operational Details: This is an internal helper function which is invoked from _calcInitialCentroids
                             to calculate the resultant hyper-rectangle centroids by combining the
                             hyper-rectangle centroids of the "touching nodes" of each leaf and return the modified 
                             collection of hyper-rectangle centroids to the calling function.
                             
            Algorithm:
                1. Declare an empty python dictionary to store and return the new set of hyper-rectangle centroids.
                2. for each leaf in preOrderedListofLeaves:
                    i. get the "touching nodes" of the current leaf. 
                    ii. if the current leaf node has any "touching nodes":
                        a. add the current leaf node at the beginning of the list of "touching nodes".
                        b. sort the current list of "touching nodes".
                        c. if we are seeing the sorted current list of "touching nodes" first time:
                            a. calculate the weighted average of all "hyperRectangleCentroids" of all the nodes
                                                            in current list of "touching nodes".
                            b. store the sorted current list of "touching nodes" as key and the result from previous
                                        step as the value for future use to the python dictionary declared at step 1. 
                        d. if we have already seen the combination before, that mean this group is already accounted
                                        for and we can now move to the next leaf in the preOrderedListofLeaves. 
                    iii. if the current leaf node doesn't have any "touching nodes", copy the current leaf node and
                                        its associated centroid to the python dictionary declared at step 1.
                3. end for
                4. Return the new set of hyper-rectangle centroids to the calling function.
            
    _calcInitialCentroids:
        Parameters:
            preOrderedListofLeaves: A list of leaves which was generated by traversing the initial CLTree in preorder.
            attributes: List of all the attributes in a constant order.
                
        Operational Details: This is an internal function with restricted access which is reposible for creating
                             initial centroids of all the hyper-rectangle created by leaves of original CLTree. 
                             This function performs its operation by visiting each leaf node one by one from the 
                             "preOrderedListofLeaves" list and it first calculate the difference between max and
                             min for each attribute one by one for each leaf, then divide all the amounts by 2 to 
                             get the mid point and add those to the respective min values to get the centroid of 
                             the hyper-rectangle created by the current leaf. After calculating the centroid of each
                             leaf separately we need to account for "touching nodes", we do that by invoking another
                             internal function "_accountForTouchedNodesForCentroids" with the initial list of
                             centroids and get back a modified list of centroids which is returned to the calling
                             function.
                            
                            Algorithm:
                                1. Declare an empty python dictionary named "hyperRectangleCentroids". 
                                2. for each leaf in "preOrderedListofLeaves":
                                    i. for each attribute in attributes:
                                        a. Get the min value of the current attribute of the current leaf and store it. 
                                        b. Get the max value of the current attribute of the current leaf and store it.
                                    ii. Get the vector difference between max-vector and min-vector. 
                                    iii. Calculate the centroid by adding the difference vector to the min vector. 
                                    iv. Store the calculated centroid against the leaf for further use and move-on. 
                                3. Invoke "_accountForTouchedNodesForCentroids" to account for "touching nodes" with
                                   the initial list of centroids along with "preOrderedListofLeaves" and "attributes"
                                   and get back a modified list of centroids which is returned to the calling function.
                             
    initializeCentroids:
        Parameters: 
            root: Instance of CLNode which is the root of underlying CLTree.
                
        Operational Details: This function is responsible to calculate the initial and original list of centroids and
                             return the result to the calling function. 
                             Algorithm:
                                1. Validate the "root". 
                                2. Invoke "preOrderTraversal" with the root and empty list to get the list of leaves
                                   which was generated by traversing the initial CLTree in preorder.
                                3. Get the list of attributes from the root. 
                                4. Invoke "_calcInitialCentroids" with the pre-ordered leaves and the attributes we got at
                                   step 2 and 3 respectively and get back a python dictionary which contain the initial
                                   hyper-rectangle centroids before the pruning process begins and return it to the
                                   calling function.


    **Defination**: A unit vector is a vector of length 1, sometimes also called a direction vector.To find a
                     unit vector with the same direction as a given vector, we divide by the magnitude of the vector. 
                                                                  
    _getUnitVectorOfRepresentationVectors:
        Parameters:
            representationVectors: A python dictionary, values of which are vectors. 
                
        Operational Details: This is an internal utility function with restricted access. Given a collection of vectors
                             this function converts all the underlying vectors into "unit vector" individually and 
                             return a dictionary with the same set of keys and respective unit-vectors of previously
                             provided vectors. 
        
    _recalculateRepresentationVectors:
        Parameters:
            key1: tuple of node id(s)
            representationVector1: Current representation vector of the collection of nodes provided as key1.
            key2: tuple of node id(s)
            representationVector2: Current representation vector of the collection of nodes provided as key2.
                
        Operational Details: This is an internal utility function with restricted access. Given 2 collection of 
                             node ids and their respective representation vectors this function calculates weighted
                             average of 2 represetation vectors based on number of actual data instances each
                             collection encompasses and create a combined key from "key1" and "key2" and return the
                             newly calculated weighted average of two initial vectors as the associated value of the
                             new key to the calling function. 
    
    
    _calcDataInstancesbyEachGroup:
        Parameters:
            listOfNodeIds: A list of tuples of node ids.
                
        Operational Details: This is an internal utility function with restricted access. Given a list of collection
                             of leaf node ids, this function calculates and returns how many actual data instances 
                             are there by each collection of node ids. 
        

    pruningTreeByMergingRepresentationVectors:
        Parameters:
            originalRepresentationVectors: A python dictionary containing the initial representation vectors for
                                           each initial group of leaf nodes [post accounted for "touching nodes"]. 
                                           Here representation vectors are the centroids of the hyper rectangle created 
                                           by the leaf nodes[or weighted average of centroids of "touching nodes"].
                            
            min_y: The minimum number of members required by each group of leaf nodes to be considered as an
                individual cluster.
                
        Operational Details:
            Base Algorithm:
            Agglomerative Hierarchical Clustering: It is a type of clustering algorithm which starts by treating
                each object as a singleton cluster. Next, pairs of clusters are successively merged until all
                clusters have been merged into one big cluster containing all objects. The result is a
                tree-based representation of the objects, named dendrogram.

            Quality Metric/ argmin Metric:
            2 different types Quality Metric or argmin Metric has been used in this implementation of the algorithm.
            The value of the instance level flag "balancedPrune". If True, then it indicates we care about the 
            variations among the number of cluster members of clusters. If False, then it indicate we care only about
            the quality of clusters. 
            
            Purity or Inv-Purity: intra-clusters-distance / inter-clusters-distance
            intra-cluster-distance and inter-cluster-distance are two very common quality metric is used to
            measure the quality of clusters or performance of the clustering algorithm. We always want to 
            minimize intra-cluster-distance and maximize inter-clusters-distance for a given set of clusters.
            Here we have combined 2 term together and as we always want to minimize the numerator and maximize
            the denominator, our goal would be to minimize the value of our “Quality Metric”.
            
            varPurity/varInv-Purity: Standard deviation of cluster members of clusters ^ Purity or Inv-Purity
            As we have already established that why we want to minimize "Purity or Inv-Purity". At the "balancedPrune"
            mode we have an additional constraint which is variance/standard-deviation of number data points among
            clusters. We want to decrease "Purity or Inv-Purity" and "variance or standard-deviation",. By combining
            these two, we have created a new metric which gives emphasis on both metric at the same time. At the
            begining of an experiment when do not know anything or not have enough confidence it is recommended
            to do balanced clustering. 
            

            Defense/Protection against outliers:
            In clustering there is a common occurrence of a problem that there may be few data points scattered across
            the hyper-plane which are not close to any well-formed groups. Now the question is how to handle it?
            Here in our clustering scheme, we have an instance level parameter “min_y” which indirectly controls
            the number of clusters by defining how many minimum data points we would need to identify a group as a
            well-formed cluster. We don’t want to violate the integrity of well formed clusters to include some
            outliers here and there, so one of the better way of handling this kind of scenario is form a separate
            group of outliers by the name of “DEFAULT”, in this way we are protecting integrity of the well-formed
            clusters and also creating mutually exclusive and exhaustive clusters. 
            We initiate the above-mentioned mechanism of “Protection against outliers” when the number of points
            which are not part of a well-formed group is less than the instance level parameter “min_y”. 
            This way we are eliminating any possibility that they might form a well-formed cluster by merging
            one to another. 


            This is the heart of the “PruneTreeByMergingCentroids”. At this point, our entire data space is
            divided into small groups, we have their representation vectors and another parameter, “min_y”.
            We will use the representation vectors as the source data to a hierarchical agglomerative clustering
            to group together small groups to form bigger groups. We will keep merging the small groups to form
            bigger groups until either all the remaining groups have “min_y” data points or total data points
            left to be part of bigger groups is less than “min_y”. In case of the later, we temporarily bind all
            the loose groups/data points which are not part of any qualified group and form a group.
            Then we calculate the statistics of the current settings and store it for future reference.
            Then we continue with the hierarchical agglomerative clustering until there is only 2 groups
            remaining but from now on after each iteration, we take a snapshot of the settings and the statistics
            for future reference. 
            At the end we take all the scenarios, calculate the quality metric and use gradient tolerance to
            find out the best setting and return the best way to cluster to the calling function. 

            Algorithm:
                1. Make a "deep copy" of initial representation vectors. 
                2. calculate number of data points at each small groups and make a "deep copy" of the result.
                3. Get all the representation vectors in a way so that it can be directly used my the
                   "scipy.cluster.hierarchy" for hirerchical clustering and also, create referece of the
                    data points by using keys from the copy of the "originalRepresentationVectors" created at step 1.
                    So that later when we will revisit the hirerarchical cluster tree from the bottom up manner
                    we can referce the original CLTree nodes from there. 
                4. Min-Max normalization of the data which will be used for hierarchical clustering to remove any
                   potential bias. 
                5. Hierarchical clustering on the normalized data. 
                6. declare an empty dictionary to store all the intermediate results as we are about to revisit the 
                   hierarchical cluster tree from bottom-up manner. 
                7. While there are more than 2 groups:
                    i. get the 2 vertices got combined. 
                    ii. get the actual node ids(or tuple of node ids) from the refernce has been created at step 3. 
                    iii. Create a new new combination from the 2 tuple we got back at previous step. It will be used
                        at id in to identify the combination of nodes.
                    iv. create a new refernce with the newly created id and delete previous references. 
                    v. Check if all the groups at this moment has data points more or equal to "min_y". 
                        a. if so, take a snapshot of the current groups and move on to next iteration. 
                    vi. Otherwise, get 2 lists of groups which have atleast "min_y" datapoints and groups which do not
                        respectively. 
                        a. Check if total members of the groups which do not have atleast "min_y" datapoints is
                           less than "min_y". 
                            i> if so, combine all the non-qualified groups together to form a temporary default group.
                            ii> take a snapshot of the current groups and move on to next iteration.
                        b. Otherwise move-on to the next iteration. 
                        c. end if
                    viii. end if
                8. end while
                9. Validation for no cluster found, it can happen due to bad value in parameters. 
                10. Now it is the time to revisit all the snapshots we have taken during our visit to dendogram
                    to figure out the best possible setting by the chosen metric.
                    **[argminMetric = "purity" or "varPurity", depending on the value of "balancedPrune".]**
                11. Once we have finalized a setting, we assign an unique cluster id to each group in the
                    finalized settings and also as a part of this process all the leaf nodes get assigned 
                    to a cluster id based on which group they belong to.
                12. Rerturn the best setting, final representation vectors and a map from node ids to cluster ids to the
                    calling function.        


    _assigingClusterToInternalNodes:
        Parameters:
            scenario: Finalized cluster settings in python dictionary format. Where the keys are tuples made-up with
                     CLTree node ids. 
                
        Operational Details:
            After finalizing the clusters and the leaf nodes each one encompasses, it is time to assign an unique 
            cluster id to each cluster. In addition to this, the purpose of this function is to set the cluster
            flag of each leaf node one by one and assign the cluster-id to the node attribute based on which cluster 
            the underlying leaf node is part of. At the same time create a refernce from node ids to assigned
            cluster ids and return the same to the calling function.
    
    **Defination**: In computer science, tree traversal (also known as tree search) is a form of graph traversal
                    and refers to the process of visiting (checking and/or updating) each node in a tree data
                    structure, exactly once. Such traversals are classified by the order in which the nodes are
                    visited.
                A Generic Pre-Order Traversal Algorithm:
                    1. Check if the current node is empty or null.
                    2. Display/Store the data part of the root (or current node).
                    3. Traverse the left subtree by recursively calling the pre-order function.
                    4. Traverse the right subtree by recursively calling the pre-order function.
                                                                  
                                                                  
    preOrderTraversal:
        Parameters:
            node: Instance of CLNode class. 
            preOrderedList: Current list of visted leaf nodes of undelying CLTree.
                
        Operational Details: This is an internal utility recursive function which returns the pre-order list of
                             visited **leaf nodes**. Unlike generic "Pre-Order Traversal Algorithm" this version
                             doesn't return list of all the nodes. It visits all the nodes but returns only the leaf
                             nodes in the order of traversal. 
                             
                             Algorithm:
                                    1. If the current node is a leaf node add the current node to the current list
                                         of visted leaf nodes and return the current list.
                                    2. If the current node is not a leaf node, get its children. 
                                         i. Invoke another copy of "preOrderTraversal" with the left child and current
                                            list of visited leaf nodes and get back the modified current list of
                                            visited leaf nodes. 
                                        ii. Invoke another copy of "preOrderTraversal" with the right child and current
                                            list of visited leaf nodes and get back the modified current list of
                                            visited leaf nodes.
                                        iii. Return the modified current list of visited leaf nodes to the 
                                              calling function.
                                
            
    preOrderTraversalofClusters:
        Parameters:
            node: Instance of CLNode class. 
            preOrderedList: Current list of visted leaf nodes of undelying CLTree which are part of some cluster.
                
        Operational Details: This is an internal utility recursive function which returns the pre-order list of
                            visited **leaf nodes which are part of some cluster**. Unlike generic 
                            "Pre-Order Traversal Algorithm" this version doesn't return list of all the nodes. 
                            It visits all the nodes but returns only the leaf which are part of some cluster
                            nodes in the order of traversal. 
                             
                            Algorithm:
                                      1. If the current node is part of some cluster add the current node to the
                                         current list of visted "includedInCluster" nodes and return the current list.
                                      2. If the current node is not "includedInCluster", then get its children. 
                                         i. If it is a leaf node, just return the unmodified current list of visited
                                            leaf nodes which are part of some cluster.
                                        ii. Invoke another copy of "preOrderTraversalofClusters" with the left child
                                             and current list of visited "includedInCluster" nodes and get back the 
                                             modified current list of visited "includedInCluster" nodes. 
                                        iii. Invoke another copy of "preOrderTraversal" with the right child and 
                                              current list of visited "includedInCluster" nodes and get back the 
                                              modified current list of visited "includedInCluster" nodes.
                                        iv. Return the modified current list of visited "includedInCluster" nodes to
                                             the calling function.
    
    leafPrunning:
        Parameters: 
            node: Instance of CLNode class. 
                
        Operational Details: This is an internal utility recursive function which operate in a bottom-up fashion. 
                             At each level/node this function checks if 2 childern of a parent are part of the same
                             cluster, if so we can safely assign the cluster at the parent level and by doing so, 
                             we are optimizing the search space by reducing it by one level at a time. 
    
    _whichChild:
        Parameters:
            node: Instance of CLNode class. 
                
        Operational Details: This is an internal utility function with restricted access, which, given a node,
                             return a flag to indicate if the current node is left or right child of its parent. 
        

    resetAllPruneNodes:
        Parameters: 
            node: Instance of CLNode class.
                
        Operational Details: This is an utility recursive function which resets all the changes has been made
                             at node level of the underlying CLTree as part of the current pruning process and revert
                             back to its original structure.This funtion is called when doing grid search to find
                             optimal values for the set of hyperparameters.
        

    _calcClusterMemebersVariance:
        Parameters:
            totalDataInstancesbyEachGroup: A python dictionary which contains data instances encompassed by each group
                                           of leaf nodes.
                
        Operational Details: This is an internal utility function with restricted access which is called from
                             "clusterStatistics". This function returns the mean of the number of data points 
                             accross the clusters and the variance among the number of data points accross the clusters

    clusterStatistics:
        Parameters:
            originalRepresentationVectors: Python dictionary consist of original representation vectors. Keys are the 
                                           original leaf nodes after "touching nodes" are accounted for and values
                                           are respective representation vectors before
                                           "pruningTreeByMergingRepresentationVectors" was invoked. 
            finalRepresentationVectors: Python dictionary consist of final representation vectors. Keys are the final
                                        groups of leaf nodes in tuple format which together forms invidual clusters 
                                        and values are respective representation vectors after
                                        "pruningTreeByMergingRepresentationVectors" operation is completed. 
            finalRepresentationVectorsToCluster: Python dictionary consist of mapping from collection of leaf nodes 
                                                 to assigned respective cluster ids.
            totalDataInstancesbyEachGroup: Python dictionary consist of mapping from group of leaf nodes and total
                                           number of data instances by each group.
                                           
                
        Operational Details:
            This function is reposible to calculate and return the key statistics including the quality metric of choice 
            which will be used to asses the quality of clustering. In this particular implementation the quality metrics
            are, purity or inv-purity, Formula: intraClusterDistance/interClusterDistance and varPurity or var-invPurity,
            Formula: Standard-deviation among the number of data points at each cluster ^ purity or inv-purity respectively 
            Apart from the quality metric this function also returns the "intra-cluster-distance",
            "inter-cluster-distance", "mean-members" accross the clusters  and "data-points" at each cluster under
            the provided setting. 
            
            Algorithm:
                1. If the instance level variable "balancedPrune" is True set the argmin metric as "varPurity" else
                   set the argmin metric as "purity". 
                2. Get the unit vector of each original and final representation vectors. 
                3. Create a map/dictionary for each final individual group of leaf nodes which was created 
                   by merging a subset of original set of leaf nodes. It would be useful at the time of calulating 
                   "intra-cluster-distance".
                4. Create all possible pair of cluters. It would be useful at the time of
                   calulating "inter-cluster-distance".
                5. Invoke an internal function named, "_calcClusterStatistics" to get the "inter-cluster-distance"  and
                   "intra-cluster-distance".
                6. calculate the purity or inv-purity by "intra-cluster-distance"/"inter-cluster-distance".
                7. Invoke "_calcClusterMemebersVariance" to get the mean of the number of data points accross the
                   clusters and the variance among the number of data points accross the clusters.
                8. Calculate "varPurity" using the information at step 7 and 6.
                9. Put all the statistics together in "result".
                10. Add "argminMetric" in the "result", according to the value of argminMetric decided at step 1. 
                11. Return the result to the calling function.
        

    _calcClusterStatistics:
        Parameters:
            allCombosOfFinalRepresentationVectors: All possible pair of cluters keys. It would be useful at the time of
                                                   calulating "inter-cluster-distance"
            finalRepresentationVectors: Python dictionary consist of final representation vectors. Keys are the final
                                        groups of leaf nodes in tuple format which together forms invidual clusters 
                                        and values are respective representation vectors after
                                        "pruningTreeByMergingRepresentationVectors" operation is completed.
            originalRepresentationVectors: Python dictionary consist of original representation vectors. Keys are the 
                                            original leaf nodes after "touching nodes" are accounted for and values
                                            are respective representation vectors before
                                            "pruningTreeByMergingRepresentationVectors" was invoked.
            finalToOriginalRepresentationVectorsMap: A python dictionary for each final individual group of leaf nodes
                                                     which was created by merging a subset of original set of
                                                     leaf nodes. It would be useful at the time of calulating 
                                                     "intra-cluster-distance".
                
        Operational Details: 
            This is an internal function which is called from "clusterStatistics", is responsible to 
             do the actual calculations required to calculate various cluster statistics including
             "intra-cluster-distance" and "inter-cluster-distance" and return those to the calling function.
            


    _calcEuclidianDistance:
        Parameters:
            a: vector 1.
            b: vector 2.
                
        Operational Details:
            This is an utility function which calculates and returns the euclidian distance bewteen 2 vectors. 
    
    getClusterID:      
        Operational Details:
            This function increment and get the current value of an instance level variable and then concatenate
            the value with some predefined string to create and the return an unique id for a cluster.
    
    pruningRedundantNodes:
        Parameters:
            root: The root of the instance level CLTree.
                
        Operational Details:
            This is an utility function. Purpose of this function is to make the search space of CLTree more efficient
            after "leafPrunning" has happened. At core CLTree devides the dataset at any level like a binary search tree.
            Suppose 2 leaf nodes are 1 generation apart, both are the same type of child of their respective parent and
            at both respective parent level the cut has happend on the same attribute, then we can safely remove 
            one level from the existing CLTree. 
            **Note**: This method changes the structure of the core CLTree, so after this method is invoked the
                      underlying CLTree can't be reused in grid-search or restored back to its original form.
                      This method/function therefore should be only called during the final and permanent pruning
                      process. 
            Each leaf node in CLTree holds an unique set of data, as we are removing one level, we would need to merge
            the data of respective leaf node to the data set of the other leaf node which will represent the both leaves
            together. 
            This is a bottom-up iterative procedure. 
        
    _recalculateDepthOfEachNodes:
        Parameters:
            node: CLNode instance. Started with CLTree root. 
            depth: Current Depth of the node. Started woth 0.
                
        Operational Details:
            This is a recursive procedure, the purpose of the procedure is recalculate the depth of all the nodes
            of the CLTree after it has gone through some structural changes. It updates the depth at the node level.
        
    _merging2Datasets:
        Parameters:
            dataset1: Instance of CLTreeModules.Data class associated with one CLNode instance.
            dataset2: Instance of CLTreeModules.Data class associated with another CLNode instance.
                
        Operational Details:
            This is an internal function called from "pruningRedundantNodes" to merge dataset of 2 nodes together when 
            reducing a level. Some basic validation is requiered before the merge. Ex.: Order of the columns
            name of all the columns and data-types of all the columns need to be same among 2 participating dataset. 
            If all the validations are fruitful then we create a separate instance of CLTreeModules.Data class 
            by using the combined dataset of dataset1 and dataset2 and return the newly created instance back to
            calling function.

pruneByGridSearch_Centroid:
    Parameters:
        cltree: The original CLTree instance on which the whole puning mechanism will be performed.
        min_y: The "min_y" dictates the minimum number of data points required for a group to be considered as a cluster.
        data_length: Total number of the data points on which the clustering is happening.
        prefixString: predefined string for cluster id.
        balancedPrune: It is a boolean flag, value of which indicates if we want to create a "balanced" clusters
                       in terms of number of data points each cluster get.
            
    Operational Details: This is method to grid search on various parameter values required for creating cluster 
                         by pruning the initial CLTree or merging the leaf nodes to form clusters.
                         In this implementation, "min_y" is the only manually provided value and also a 
                         hyper parameter. To get the most optimal value of "min_y" which will provide the 
                         best value for the cost function, we do a grid search on 10 equally spaced values of 
                         "min_y" starting from "min_y" - 10% of "min_y" to "min_y". At each value we create a 
                         instance of "PruneTreeByMergingCentroids" with the original "cltree" and current value of 
                         "min_y". At the end of each pruning procedure, we store the current
                         qulaity metric with the current setting for future refernce. (Here in this implementation
                         the cost/quality metric is "purity"[actually inverse purity.] or 
                         "varPurity" [actually std. ^ inverse purity.], which is we are trying to minimize.)
                         Then move to the new value of "min_y". At the end when we are done with testing all the 
                         potential values of "min_y". We take the value of "min_y" where the quality metric is best
                         to the final pruning with "searchMode" set to False. We set the base version of the pruning
                         algorithm and return the base version and the result to the calling function. After the final
                         pruning all the required changes has happened to the CLTree and CLNode level. 